In [66]:
import polars as pl
from omegaconf import OmegaConf

from src.utilities import find_project_root

In [67]:
project_root = find_project_root()

datetime = [
    # date
    [2024, 12, 20],
    # time
    [11, 18, 00],
]
date = "-".join((f"{i:02}" for i in datetime[0]))
time = "-".join((f"{i:02}" for i in datetime[1]))

csv_path = (
    project_root
    / "reports"
    / "features"
    / "ClassifierReflectance"
    / date
    / time
)
cfg = OmegaConf.load(csv_path / ".hydra/config.yaml")

In [31]:
print(csv_path)
print(list(csv_path.parent.iterdir()))
csv_path.exists()

/home/shinsei/MyResearches/feat_extrc/reports/features/ClassifierReflectance/2024-12-20/11-18-00
[PosixPath('/home/shinsei/MyResearches/feat_extrc/reports/features/ClassifierReflectance/2024-12-20/11-18-00')]


True

In [43]:
check_data = pl.read_csv(csv_path / "features_check_data.csv")
check_data

column_0,column_1,column_2,column_3,column_4,column_5,column_6,column_7,column_8,column_9,column_10,column_11,column_12,column_13,column_14,column_15,column_16,column_17,column_18,column_19,column_20,column_21,column_22,column_23,column_24,column_25,column_26,column_27,column_28,column_29,column_30,column_31,target,prediction,dirname,filename
f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,i64,i64,i64,str
-0.046268,0.054592,-0.041591,-1.228894,-1.290422,-0.067401,0.039068,-1.918401,-1.838542,-1.710984,0.055068,0.020493,-0.053506,0.054571,0.299959,-0.023875,-0.051314,0.027861,0.048626,-0.891681,-1.423253,0.043976,-1.901998,-0.00717,0.055137,-0.002293,-0.05102,0.035556,-0.063491,-0.04865,-0.049328,-0.053537,0,0,3,"""10k_3_h000_w00…"
-0.023983,0.031178,-0.033927,-0.933908,-0.944822,-0.034344,0.018034,-1.398027,-1.374643,-1.304216,0.031339,0.025587,-0.030518,0.030419,0.226531,-0.006737,-0.028672,0.009566,0.026007,-0.679945,-1.087599,0.022336,-1.391321,-0.015222,0.033053,-0.008529,-0.028354,0.015291,-0.037257,-0.026106,-0.026739,-0.037437,0,0,3,"""10k_3_h000_w00…"
-0.132207,0.141845,-0.076364,-3.039994,-3.256327,-0.195156,0.1225214,-4.905235,-4.73561,-4.321697,0.143351,0.01662,-0.139577,0.144563,0.7018597,-0.097554,-0.137027,0.104567,0.135038,-2.122927,-3.640405,0.128205,-4.868007,0.043543,0.137526,0.036652,-0.136899,0.117209,-0.15879,-0.134818,-0.135457,-0.11299,0,0,3,"""10k_3_h000_w00…"
-0.002074,0.007586,-0.017435,-0.67372,-0.682094,-0.005548,-0.002076,-1.017705,-0.973552,-0.939652,0.007594,0.023005,-0.007188,0.006638,0.1773721,0.008809,-0.005736,-0.007271,0.003562,-0.497148,-0.778641,0.001026,-1.007586,-0.010214,0.009668,-0.012984,-0.005468,-0.003851,-0.01151,-0.00368,-0.004173,-0.017482,0,0,3,"""10k_3_h000_w00…"
-0.151679,0.161419,-0.08532,-3.475654,-3.707442,-0.225153,0.1417101,-5.663592,-5.465346,-4.957856,0.163066,0.007437,-0.15898,0.164598,0.78948,-0.115733,-0.156395,0.123036,0.154564,-2.398564,-4.188815,0.147412,-5.618719,0.060906,0.156277,0.050917,-0.156301,0.136196,-0.17983,-0.154296,-0.154921,-0.126999,0,0,3,"""10k_3_h000_w01…"
-0.038082,0.046204,-0.041146,-1.037937,-1.062173,-0.053285,0.031204,-1.582998,-1.531613,-1.439649,0.046506,0.021158,-0.045314,0.045753,0.253082,-0.017269,-0.043183,0.020869,0.040388,-0.756423,-1.193697,0.036039,-1.567517,-0.013462,0.047524,-0.004836,-0.042861,0.027927,-0.053739,-0.04044,-0.041127,-0.048763,0,0,3,"""10k_3_h000_w01…"
-0.104319,0.113597,-0.067302,-2.465059,-2.611792,-0.15366,0.095468,-3.966304,-3.83213,-3.496198,0.114696,0.0150274,-0.111777,0.11525,0.572481,-0.074153,-0.109323,0.080008,0.107015,-1.720347,-2.944612,0.100927,-3.930194,0.029449,0.11117,0.026478,-0.109126,0.090804,-0.127688,-0.106887,-0.107555,-0.094784,0,0,3,"""10k_3_h000_w01…"
-0.140286,0.150358,-0.081546,-3.156514,-3.374769,-0.206024,0.130037,-5.070954,-4.916175,-4.495833,0.151941,0.019148,-0.147993,0.1532221,0.7237281,-0.103409,-0.145353,0.110912,0.143269,-2.207557,-3.780224,0.136074,-5.034639,0.042714,0.145827,0.037839,-0.145219,0.124392,-0.168071,-0.143036,-0.143708,-0.119554,0,0,3,"""10k_3_h000_w02…"
-0.077338,0.086328,-0.05566,-1.888766,-1.998134,-0.113534,0.069184,-3.017598,-2.898673,-2.6596,0.087135,0.016554,-0.084829,0.08719,0.447121,-0.050622,-0.082434,0.055627,0.079914,-1.33486,-2.230525,0.07443,-2.987274,0.01284,0.085298,0.013108,-0.082188,0.065029,-0.098071,-0.079851,-0.080532,-0.076063,0,0,3,"""10k_3_h000_w02…"


In [42]:
check_data_clsf_failes = check_data.filter(
    pl.col("target") != pl.col("prediction")
).select(pl.exclude(r"^column_\d+$"))

train_data = pl.read_csv(csv_path / "features_train_data.csv")
train_data_clsf_failes = train_data.filter(
    pl.col("target") != pl.col("prediction")
).select(pl.exclude(r"^column_\d+$"))

In [54]:
check_data.groupby(by=pl.col("target")).agg(pl.col("dirname").unique()).explode("dirname")

target,dirname
i64,i64
0,3
0,5
0,7
1,4
1,6
1,8


In [52]:
check_data_clsf_failes.filter(pl.col("target")==1)

target,prediction,dirname,filename
i64,i64,i64,str
1,0,4,"""10k_4_h352_w09…"
1,0,4,"""10k_4_h800_w12…"
1,0,4,"""10k_4_h896_w12…"
1,0,6,"""10k_6_h032_w04…"
1,0,6,"""10k_6_h256_w00…"
1,0,6,"""10k_6_h288_w06…"
1,0,6,"""10k_6_h352_w10…"
1,0,6,"""10k_6_h544_w01…"
1,0,6,"""10k_6_h640_w02…"


In [71]:
cfg.dataset

{'image_size': 32, 'image_target': 'CNTForest', 'train_path': 'data/processed/CNTForest/cnt_sem_${dataset.image_size}x${dataset.image_size}/10k', 'check_path': 'data/processed/check/CNTForest/cnt_sem_for_check_${dataset.image_size}x${dataset.image_size}/10k', 'cls_conditions': {0: ['3', '5', '7'], 1: ['4', '6', '8']}, 'transform': {'Grayscale': 1, 'ToTensor': 0}}

In [85]:
from PIL import Image

dataset_dir = (
    project_root
    / "data"
    / "processed"
    / "CNTForest"
    / f"cnt_sem_{cfg.dataset.image_size}x{cfg.dataset.image_size}"
    / "10k"
)
print(dataset_dir.exists())

images: dict[int, list[Image.Image]] = {}

unique_dirname = check_data_clsf_failes.filter(
    pl.col("target") == 0
).select(
    pl.col("dirname")
).unique("dirname").to_series()
for i in unique_dirname.sort():
    paths = check_data_clsf_failes.filter(
        pl.col("dirname") == i
    ).select(
        (
            pl.col("dirname").cast(pl.Utf8) + pl.lit("/") + pl.col("filename")
        ).alias("path"),
    ).to_series()
    images[i] = [Image.open(dataset_dir / p).convert("L") for p in paths]

True


In [83]:
images

{3: [<PIL.PngImagePlugin.PngImageFile image mode=RGB size=32x32>,
  <PIL.PngImagePlugin.PngImageFile image mode=RGB size=32x32>,
  <PIL.PngImagePlugin.PngImageFile image mode=RGB size=32x32>,
  <PIL.PngImagePlugin.PngImageFile image mode=RGB size=32x32>,
  <PIL.PngImagePlugin.PngImageFile image mode=RGB size=32x32>,
  <PIL.PngImagePlugin.PngImageFile image mode=RGB size=32x32>,
  <PIL.PngImagePlugin.PngImageFile image mode=RGB size=32x32>,
  <PIL.PngImagePlugin.PngImageFile image mode=RGB size=32x32>],
 5: [<PIL.PngImagePlugin.PngImageFile image mode=RGB size=32x32>,
  <PIL.PngImagePlugin.PngImageFile image mode=RGB size=32x32>,
  <PIL.PngImagePlugin.PngImageFile image mode=RGB size=32x32>],
 7: [<PIL.PngImagePlugin.PngImageFile image mode=RGB size=32x32>,
  <PIL.PngImagePlugin.PngImageFile image mode=RGB size=32x32>,
  <PIL.PngImagePlugin.PngImageFile image mode=RGB size=32x32>,
  <PIL.PngImagePlugin.PngImageFile image mode=RGB size=32x32>,
  <PIL.PngImagePlugin.PngImageFile image mod